In [0]:
#Sklearn
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

#PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
class PVDATATIME(Dataset):
  def __init__(self, csv_path, norm = True):
    if norm == True:
      from sklearn.preprocessing import StandardScaler
      self.data = pd.read_csv(csv_path)
      scaler = StandardScaler()
      self.data = scaler.fit_transform(self.data)
    else:
      self.data = pd.read_csv(csv_path)

  def __getitem__(self,idx):
    features = self.data[idx][0:10]
    label = self.data[idx][-1:]

    #Tensor

    features = torch.from_numpy(features.astype(np.float32))
    label = torch.from_numpy(label.astype(np.float32))

    return features, label

  def __len__(self):
    return len(self.data)

## Rocha


In [0]:
class RochaANN(nn.Module):
  def __init__(self, in_f,h1 = 30):
    super(RochaANN, self).__init__()

    #In_f -> h1 (Firt Connect)
    self.fc1 = nn.Linear(in_f,h1)
    
    # h1 -> out (2°)
    self.out = nn.Linear(h1, 1)


  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.out(x)
    return x

In [0]:
train_path = '/content/drive/My Drive/IC Victor/Projeto EUA/1 minuto/Banco de dados/pv_1min_train'
test_path = '/content/drive/My Drive/IC Victor/Projeto EUA/1 minuto/Banco de dados/pv_1min_test'

train_dataset = PVDATATIME(train_path)
test_dataset = PVDATATIME(test_path)

print(f'Q. de dados para treino: {train_dataset.__len__()}')
print(f'Q. de dados para teste: {test_dataset.__len__()}')

Q. de dados para treino: 1028982
Q. de dados para teste: 522720


In [0]:
train_loader = DataLoader(train_dataset,
                          batch_size = 5000,
                          shuffle = False,
                          num_workers = 4) 

test_loader = DataLoader(test_dataset,
                         batch_size = 5000,
                         shuffle = False,
                         num_workers = 4)

In [0]:
#Inicializando criterio, otimizador e rede.
device = torch.device('cuda')
epochs = 500

in_f = len(train_dataset[0][0])
h1 = 30

model = RochaANN(in_f,h1).to(device)

criterion = nn.MSELoss().to(device)


optimizer= torch.optim.SGD(model.parameters(),
                             lr = 1e-3,
                             momentum = 0.9,
                             weight_decay = 5e-4)

In [0]:
#Treino

model.train()
training_loss = []
optimizer.zero_grad()
start_time = time.time()
for epoch in range(epochs):
  print(f'Epoca : {epoch}')

  for b, (X_train, Y_train) in enumerate(train_loader):
    #Cast dos dados.
    X_train = X_train.to(device)
    Y_train = Y_train.to(device)

    #Treino
    y_pred = model(X_train)
    loss = torch.sqrt(criterion(y_pred,Y_train))

    training_loss.append(loss.cpu().data)


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  
    
    if b % 250 == 0:
      print(f'\t Training batch {b} \t loss: {loss.cpu().data}')

    #if epoch % 1000 == 0:
      #torch.save(model.state_dict(),'/content/drive/My Drive/IC Victor/Projeto EUA/1 minuto/Rocha1min.pt')



final_time = time.time() - start_time

training_loss = np.asarray(training_loss)

print(f'Tempo:{final_time / 60} minutos')

Epoca : 0
	 Training batch 0 	 loss: 1.1500664949417114
Epoca : 1
	 Training batch 0 	 loss: 0.6167130470275879
Epoca : 2
	 Training batch 0 	 loss: 0.48651230335235596
Epoca : 3
	 Training batch 0 	 loss: 0.45792239904403687
Epoca : 4
	 Training batch 0 	 loss: 0.4492005705833435
Epoca : 5
	 Training batch 0 	 loss: 0.4435329735279083
Epoca : 6
	 Training batch 0 	 loss: 0.4386596381664276
Epoca : 7
	 Training batch 0 	 loss: 0.4341835677623749
Epoca : 8
	 Training batch 0 	 loss: 0.42999398708343506
Epoca : 9
	 Training batch 0 	 loss: 0.425997257232666
Epoca : 10
	 Training batch 0 	 loss: 0.4221288561820984
Epoca : 11
	 Training batch 0 	 loss: 0.41827160120010376
Epoca : 12
	 Training batch 0 	 loss: 0.4144991338253021
Epoca : 13
	 Training batch 0 	 loss: 0.41082441806793213
Epoca : 14
	 Training batch 0 	 loss: 0.40718379616737366
Epoca : 15
	 Training batch 0 	 loss: 0.40356647968292236
Epoca : 16
	 Training batch 0 	 loss: 0.4001843333244324
Epoca : 17
	 Training batch 0 	 los

In [0]:
#Teste
#Load Model
model.load_state_dict(torch.load('/content/drive/My Drive/IC Victor/Projeto EUA/1 minuto/Rocha1min.pt'))

print('Modelo carregado com sucesso!')

test_loss = []
score = []

with torch.no_grad():
  model.eval()
  print('Inicio do teste')
  for b, (X_test, y_test) in enumerate(test_loader):

    X_test = X_test.to(device)
    y_test = y_test.to(device)

    y_val = model(X_test)
    loss = torch.sqrt(criterion(y_val,y_test))

    test_loss.append(loss.cpu().data)

    score.append(r2_score(y_test.cpu().numpy(),y_val.cpu().numpy()))

  test_loss = np.asarray(test_loss)
  score = np.asarray(score)

  print(f'RMSE: {test_loss.mean()}, \n Score: {score.mean()}')


Modelo carregado com sucesso!
Inicio do teste
RMSE: 0.45149466395378113, 
 Score: 0.0753820122359974


In [0]:
0,45149466395378113, 
 Score: 0,0753820122359974

## Adel

In [0]:
class AdelANN(nn.Module):
  def __init__(self, in_f,h1 = 11, h2 = 17, h3 = 24):
    super(AdelANN, self).__init__()

    #In_f -> h1 (Firt Connect)
    self.fc1 = nn.Linear(in_f,h1)

    # h1 -> h2 (2°)
    self.fc2 = nn.Linear(h1, h2)

    #h2 -> h3(3°)
    self.fc3 = nn.Linear(h2,h3)

    #h3 -> out (4°)
    self.out = nn.Linear(h3,1)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))


    x = self.out(x)
    return x

In [0]:
#Inicializando criterio, otimizador e rede.
device = torch.device('cuda')
epochs = 1000

in_f = len(train_dataset[0][0])
h1 = 11
h2 = 17
h3 = 24

model = AdelANN(in_f,h1,h2,h3).to(device)

criterion = nn.MSELoss().to(device)


optimizer= torch.optim.SGD(model.parameters(),
                             lr = 1e-3,
                             momentum = 0.9,
                             weight_decay = 5e-4)

In [0]:
model.train()
training_loss = []
optimizer.zero_grad()
start_time = time.time()
for epoch in range(epochs):
  print(f'Epoca : {epoch}')

  for b, (X_train, Y_train) in enumerate(train_loader):
    #Cast dos dados.
    X_train = X_train.to(device)
    Y_train = Y_train.to(device)

    #Treino
    y_pred = model(X_train)
    loss = torch.sqrt(criterion(y_pred,Y_train))

    training_loss.append(loss.cpu().data)


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  
    
    if b % 250 == 0:
      print(f'\t Training batch {b} \t loss: {loss.cpu().data}')

    if epoch % 1000 == 0:
      torch.save(model.state_dict(),'/content/drive/My Drive/IC Victor/Projeto EUA/1 minuto/Adel1min.pt')



final_time = time.time() - start_time

training_loss = np.asarray(training_loss)

print(f'Tempo:{final_time / 60} minutos')

Epoca : 0
	 Training batch 0 	 loss: 0.9970700740814209
Epoca : 1
	 Training batch 0 	 loss: 0.9582991600036621
Epoca : 2
	 Training batch 0 	 loss: 0.8842318654060364
Epoca : 3
	 Training batch 0 	 loss: 0.6809827089309692
Epoca : 4
	 Training batch 0 	 loss: 0.5094010829925537
Epoca : 5
	 Training batch 0 	 loss: 0.4588529169559479
Epoca : 6
	 Training batch 0 	 loss: 0.441040575504303
Epoca : 7
	 Training batch 0 	 loss: 0.433590292930603
Epoca : 8
	 Training batch 0 	 loss: 0.42972531914711
Epoca : 9
	 Training batch 0 	 loss: 0.42670103907585144
Epoca : 10
	 Training batch 0 	 loss: 0.42388999462127686
Epoca : 11
	 Training batch 0 	 loss: 0.4208980202674866
Epoca : 12
	 Training batch 0 	 loss: 0.417499840259552
Epoca : 13
	 Training batch 0 	 loss: 0.4136006236076355
Epoca : 14
	 Training batch 0 	 loss: 0.40943101048469543
Epoca : 15
	 Training batch 0 	 loss: 0.4049932658672333
Epoca : 16
	 Training batch 0 	 loss: 0.40012967586517334
Epoca : 17
	 Training batch 0 	 loss: 0.39

In [0]:
#Load Model
model.load_state_dict(torch.load('/content/drive/My Drive/IC Victor/Projeto EUA/1 minuto/Adel1min.pt'))

print('Modelo carregado com sucesso!')

test_loss = []
score = []

with torch.no_grad():
  model.eval()
  print('Inicio do teste')
  for b, (X_test, y_test) in enumerate(test_loader):

    X_test = X_test.to(device)
    y_test = y_test.to(device)

    y_val = model(X_test)
    loss = torch.sqrt(criterion(y_val,y_test))

    test_loss.append(loss.cpu().data)

    score.append(r2_score(y_test.cpu().numpy(),y_val.cpu().numpy()))

  test_loss = np.asarray(test_loss)
  score = np.asarray(score)

  print(f'RMSE: {test_loss.mean()}, \n Score: {score.mean()}')



Modelo carregado com sucesso!
Inicio do teste
RMSE: 0.9576559662818909, 
 Score: -3.2951310969931993


In [0]:
0,9576559662818909, 
 Score: -3,2951310969931993